# Development platform for SPICE-based orbit propagator

2019/11/07

### Developed with problem for GTOC4 in mind:
- allow mutliple bodies to be propagated at the same time (e.g. multiple asteroids and spacecraft)
- compute relative distance between specified bodies during propagation)

In [6]:
import numpy as np
import spiceypy as spice
import pandas as pd

In [7]:
# load spice kernels
spice.furnsh('../spice/naif0012.tls')

# define gravitational parameters
MU = 1.32712440018*10**11   # sun's gravitational parameter [km^3/s^2]
AU = 1.49597870691*10**8    # astronomical unit [AU] to [km]
g0 = 9.80665                # gravitational acceleration [m/s^2]


In [33]:
# import ephemerides data from GTOC4 into pandas dataframe
elmnts = ['Name','Epoch','a','e','i','LAN','periap','theta']
df = pd.read_fwf('../gtoc4_problem_data.txt', sep='\t', skiprows=1)
# reassign column names
df.columns =  ['Name','Epoch','a','e','i','LAN','omega','M0']
num,tmp = df.shape
print(df.shape)
df.head(3)

(1436, 8)


,Name,Epoch,a,e,i,LAN,omega,M0
0,'1580',54800.0,2.196803,0.487683,52.090794,62.324795,159.539839,3.521687
1,'1620',54800.0,1.245551,0.335511,13.337693,337.266017,276.806539,136.950031
2,'1943',54800.0,1.430317,0.255816,8.704065,246.401915,338.253875,288.405238


In [41]:
# create smaller pandas dataframe to test propagator

# FIXME : create here spacecraft dataframe, then merge?
#  df1.merge(df2, left_on='lkey', right_on='rkey')
el_pd1 = df[0:2]
el_pd1

,Name,Epoch,a,e,i,LAN,omega,M0
0,'1580',54800.0,2.196803,0.487683,52.090794,62.324795,159.539839,3.521687
1,'1620',54800.0,1.245551,0.335511,13.337693,337.266017,276.806539,136.950031


In [42]:
# conversion between MJD and JD
# (Data given in MJD, SPICE deals with JD)
def _mjd2jd(mjd):
    """function converts MJD (Modified Julian Date) to JD (Julian Date)
    Args:
        mjd (float or lst): float or list of floats of ephemerides in MJD to be converted to JD
    Returns:
        (float or float): float or list of ephemeris equivalent in JD
    """
    if type(mjd) == list: 
        jd = [el + 2400000.5 for el in mjd]
    else:
        jd = mjd + 2400000.5
    return jd

def _jd2mjd(jd):
    """function converts JD (Julian Date) to MJD (Modified Julian Date)
    Args:
        mjd (float or lst): float or list of floats of ephemerides in JD to be converted to MJD
    Returns:
        (float or float): float or list of ephemeris equivalent in MJD
    """
    if type(mjd) == list: 
        mjd = [el - 2400000.5 for el in jd]
    else:
        mjd = jd - 2400000.5
    return jd


In [ ]:
# propagator ... could also make a separate function just to compute dr? or an option?
def propagate_SPICE(etr_MJD, timestep=5000, elpd):
    """
    SPICE-powered orbit propagator developed for GTOC4
    Args:
        et_MJD (lst): list including start and end time of propagation (in MJD)
        timestep (float): time step of propagation [sec]
        elpd (pandas df): pandas dataframe of orbital elements to be propagated (expect spacecraft to be first row)
    Returns:
        FIXME : need to output: dr, points
    """
    
    # convert time range from MJD to JD
    etr_JD = _mjd2jd((etr_MJD))
    # create time array
    etrsteps = [x * (etr_jd[1] - etr_jd[1])/timestep + etr_jd[0] for x in range(timestep)]
    
    # prepare orbital elements
    state = spice.conics(elts, et0)
    
    
    

In [ ]:
# test run function with el_pd1
# propagation range time
etr_utc = ['Jan 1, 2015', 'Jun 31, 2015']  # currently working with 6 months ~ 182 days
etr_MJD = [spice.str2et(etr_utc[0]), spice.str2et(etr_utc[1])]
